In [ ]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB, quicksum

from constants import (
    MAX_TAXI_ZONE_ID,
    location_ids,
    excluded_location_ids,
    location_id_to_index,
    num_locations,
    taxi_type,
    YEARS,
    MONTHS
)

$$\begin{align*}
\max_{\bar{e}, \bar{f}, \bar{a}} \quad & \frac{1}{K\Delta} \sum_{k=0}^{K-1} \sum_{i=1}^r \sum_{j=1}^r \bar{a}_i \lambda_i(t + k\Delta) P_{ij}(t + k\Delta) c_{ij}(t + k\Delta) \cdot\Delta \\
\text{subject to} \quad 

& \frac{1}{K\Delta} \sum_{k=0}^{K-1} \lambda_i(t + k\Delta) P_{ij}(t + k\Delta) \bar{a}_i \cdot \Delta  = \frac{1}{K\Delta} \sum_{k=0}^{K-1} \mu_{ij}(t + k\Delta) \bar{f}_{ij} \cdot \Delta \\

& \frac{1}{K\Delta} \left(\sum_{k=0}^{K-1} \mu_{ij}(t + k\Delta) \cdot \Delta\right) \bar{e}_{ij} \leq \sum_{j=1}^r \left(\frac{1}{K\Delta} \sum_{k=0}^{K-1} \mu_{ji}(t + k\Delta)  \cdot \Delta \right)\bar{f}_{ji}\\

& \sum_{\substack{j=1 \\ j \neq i}}^r \frac{1}{K\Delta} \left(\sum_{k=0}^{K-1} \mu_{ji}(t + k\Delta) \cdot \Delta\right) \bar{e}_{ji} \leq \frac{1}{K\Delta}\left(\sum_{k=0}^{K-1} \lambda_i(t + k\Delta)\cdot \Delta \right) \bar{a}_i \\

& \frac{1}{K\Delta} \left(\sum_{k=0}^{K-1} \lambda_i(t + k\Delta)\cdot \Delta \right) \bar{a}_i  \\

&\quad\leq \sum_{\substack{j=1 \\ j \neq i}}^r \left(\frac{1}{K\Delta} \sum_{k=0}^{K-1} \mu_{ji}(t + k\Delta)  \cdot \Delta \right)\bar{e}_{ji} + \sum_{j=1}^r \left(\frac{1}{K\Delta} \sum_{k=0}^{K-1} \mu_{ji}(t + k\Delta) \cdot \Delta \right)\bar{f}_{ji} \\

& \frac{1}{K\Delta} \left(\sum_{k=0}^{K-1} \lambda_i(t + k\Delta)\cdot \Delta \right) \bar{a}_i + \sum_{\substack{j=1 \\ j \neq i}}^r \left(\frac{1}{K\Delta} \sum_{k=0}^{K-1} \mu_{ij}(t + k\Delta)  \cdot \Delta \right)\bar{e}_{ij} \\

&\quad = \sum_{\substack{j=1 \\ j \neq i}}^r \left(\frac{1}{K\Delta} \sum_{k=0}^{K-1} \mu_{ji}(t + k\Delta)  \cdot \Delta \right)\bar{e}_{ji} + \sum_{j=1}^r \left(\frac{1}{K\Delta} \sum_{k=0}^{K-1} \mu_{ji}(t + k\Delta) \cdot \Delta \right)\bar{f}_{ji} \\
& e_{ij}, f_{ij} \in [0, 1], \sum_{i=1}^r \sum_{j=1}^r e_{ij} + f_{ij} = 1\\
& 0 \leq \bar{a}_i \leq 1
\end{align*}
$$

In [ ]:
Delta = 20 # in minutes
T_max = int(24 * (60 // Delta))

In [3]:
with np.load('trip_counts.npz') as data:
    trip_counts = data['trip_counts']
    num_dates = data['num_dates']

with np.load('mu.npz') as data:
    mu = data['mu']
    
lambda_ = trip_counts.sum(axis=2) / (Delta / 60 * num_dates)

P = trip_counts / trip_counts.sum(axis=2, keepdims=True)

In [4]:
def solve_routing(lambda_, mu, P, Delta, t_start=0, K=6):
    r = lambda_.shape[1]
    model = gp.Model("lookahead_empty_car_routing")
    
    # Variables
    a = model.addVars(r, lb=0, ub=1, name="a")
    e = model.addVars(r, r, lb=0, ub=1, name="e")
    f = model.addVars(r, r, lb=0, ub=1, name="f")
    
    # Objective
    model.setObjective(
        (1/K) * quicksum(
            quicksum(
                quicksum(a[i] * lambda_[t_start + k, i] * P[t_start + k, i, j] * Delta
                         for j in range(r))
                for i in range(r))
            for k in range(K)),
        GRB.MAXIMIZE
    )
    
    # Constraint: ride flow balance (eq. 10a)
    for i in range(r):
        for j in range(r):
            lhs = (1/(K*Delta)) * quicksum(lambda_[t_start + k, i] * P[t_start + k, i, j] * a[i] * Delta for k in range(K))
            rhs = (1/(K*Delta)) * quicksum(mu[t_start + k, i, j] * f[i,j] * Delta for k in range(K))
            model.addConstr(lhs == rhs, name=f"ride_flow_balance_{i}_{j}")
    
    # Constraint: empty car flow balance (eq. 10b)
    for i in range(r):
        for j in range(r):
            if i != j:
                lhs = (1/(K*Delta)) * quicksum(mu[t_start + k, i, j] * e[i,j] * Delta for k in range(K))
                rhs = (1/(K*Delta)) * quicksum(mu[t_start + k, j, i] * f[j,i] * Delta for k in range(K))
                model.addConstr(lhs <= rhs, name=f"empty_car_flow_{i}_{j}")
    
    # Constraint: supply conservation lower bound (eq. 10c, first inequality)
    for i in range(r):
        lhs = quicksum(
            (1/(K*Delta)) * quicksum(mu[t_start + k, j, i] * e[j,i] * Delta for k in range(K))
            for j in range(r) if j != i
        )
        rhs = (1/(K*Delta)) * quicksum(lambda_[t_start + k, i] * a[i] * Delta for k in range(K))
        model.addConstr(lhs <= rhs, name=f"supply_lower_{i}")
    
    # Constraint: supply conservation upper bound (eq. 10c, second inequality)
    for i in range(r):
        lhs = (1/(K*Delta)) * quicksum(lambda_[t_start + k, i] * a[i] * Delta for k in range(K))
        rhs = quicksum(
            (1/(K*Delta)) * quicksum(mu[t_start + k, j, i] * e[j,i] * Delta for k in range(K))
            for j in range(r) if j != i
        ) + \
        quicksum(
            (1/(K*Delta)) * quicksum(mu[t_start + k, j, i] * f[j,i] * Delta for k in range(K))
            for j in range(r)
        )
        model.addConstr(lhs <= rhs, name=f"supply_upper_{i}")
    
    # Constraint: Car flow balance (eq. 10d)
    for i in range(r):
        lhs = (1/(K*Delta)) * quicksum(lambda_[t_start + k, i] * a[i] * Delta for k in range(K)) + \
              quicksum(
                  (1/(K*Delta)) * quicksum(mu[t_start + k, i, j] * e[i,j] * Delta for k in range(K))
                  for j in range(r) if j != i
              )
        rhs = quicksum(
            (1/(K*Delta)) * quicksum(mu[t_start + k, j, i] * e[j,i] * Delta for k in range(K))
            for j in range(r) if j != i
        ) + \
        quicksum(
            (1/(K*Delta)) * quicksum(mu[t_start + k, j, i] * f[j,i] * Delta for k in range(K))
            for j in range(r)
        )
        model.addConstr(lhs == rhs, name=f"supply_lower_{i}")
    
    # Unit mass constraint
    model.addConstr(
        quicksum(e[i,j] + f[i,j] for i in range(r) for j in range(r)) == 1,
        name="unit_mass"
    )
    
    model.optimize()
    
    return model, a, e, f


In [14]:
t_start = 24
K = 4

res = solve_routing(lambda_, mu, P, Delta, t_start=t_start, K=K)
model, a, e, f = res

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 24.1.0 24B2083)

CPU model: Apple M4 Pro
Thread count: 14 physical cores, 14 logical processors, using up to 14 threads

Optimize a model with 110921 rows, 110685 columns and 552238 nonzeros
Model fingerprint: 0xec79d57e
Coefficient statistics:
  Matrix range     [2e-04, 4e+02]
  Objective range  [1e+01, 8e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 110921 rows and 110685 columns
Presolve time: 0.03s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.06 seconds (0.09 work units)
Optimal objective -0.000000000e+00


In [16]:
mu_avg = np.mean(mu[t_start:t_start+K], axis=0)
lambda_avg = np.mean(lambda_[t_start:t_start+K], axis=0)
r = len(location_ids)

def compute_q_matrix(a, e, f, mu_avg, lambda_avg):
    """
    a, e, f: dicts of Gurobi variable values: a[i], e[i,j], f[i,j]
    mu_avg: 2D array of shape (r, r) averaged over time
    lambda_avg: 1D array of shape (r,) averaged over time
    """
    r = len(a)
    q = np.zeros((r, r))

    # Precompute denominator for qij and qii
    for i in range(r):
        denom = sum(mu_avg[k, i] * f[k, i] for k in range(r))

        for j in range(r):
            if i != j:
                if denom > 0:
                    q[i, j] = mu_avg[i, j] * e[i, j] / denom
                else:
                    q[i, j] = 0.0
            else:
                # qii computation
                numerator = lambda_avg[i] * a[i] - sum(
                    mu_avg[k, i] * e[k, i] for k in range(r) if k != i
                )
                q[i, i] = numerator / denom if denom > 0 else 0.0

    return q

In [17]:
a_vals = {i: a[i].X for i in range(r)}
e_vals = {(i, j): e[i, j].X for i in range(r) for j in range(r)}
f_vals = {(i, j): f[i, j].X for i in range(r) for j in range(r)}

Q = compute_q_matrix(a_vals, e_vals, f_vals, mu_avg, lambda_avg)

In [18]:
Q

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])